# DNA-Based Biomarker Discovery and Drug Target Identification

This notebook performs comprehensive DNA sequence analysis to identify potential biomarkers and drug targets through feature extraction and machine learning.


## 1. Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, accuracy_score,
    precision_recall_curve, auc
)
from sklearn.preprocessing import StandardScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline


## 2. Load Data


In [ ]:
# Load the genomics data
df = pd.read_csv('../data/genomics_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nLabel distribution:")
print(df['Labels'].value_counts())
print(f"\nDataset info:")
print(df.info())


## 3. DNA Sequence Feature Extraction


In [ ]:
# Import feature extraction function
import sys
sys.path.append('../scripts')
from dna_feature_extraction import extract_dna_features, get_feature_names

# Extract features
print("Extracting features from DNA sequences...")
sequences = df['Sequences'].values.tolist()
X = extract_dna_features(sequences)
y = df['Labels'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Number of features: {X.shape[1]}")
feature_names = get_feature_names()
print(f"\nFeature names: {len(feature_names)} features")
print(f"First 10 features: {feature_names[:10]}")


## 4. Feature Importance Analysis


In [ ]:
# Train a Random Forest to get feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X, y)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 20 Most Important Features:")
print(feature_importance.head(20))

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(20)
sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')
plt.title('Top 20 Most Important DNA Sequence Features', fontsize=16, fontweight='bold')
plt.xlabel('Feature Importance', fontsize=12)
plt.ylabel('Feature Name', fontsize=12)
plt.tight_layout()
plt.savefig('../results/figures/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()


## 5. Data Splitting and Preprocessing


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"\nTraining label distribution:")
print(pd.Series(y_train).value_counts())
print(f"\nTest label distribution:")
print(pd.Series(y_test).value_counts())

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 6. Machine Learning Models


In [ ]:
# Define models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42, C=1.0, gamma='scale'),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1, C=1.0)
}

# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Training {name}...")
    print(f"{'='*50}")
    
    # Use scaled data for SVM and Logistic Regression
    if name in ['SVM', 'Logistic Regression']:
        X_train_use = X_train_scaled
        X_test_use = X_test_scaled
    else:
        X_train_use = X_train
        X_test_use = X_test
    
    # Train model
    model.fit(X_train_use, y_train)
    
    # Predictions
    y_pred = model.predict(X_test_use)
    y_pred_proba = model.predict_proba(X_test_use)[:, 1]
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_use, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'auc': auc_score,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"AUC-ROC: {auc_score:.4f}")
    print(f"CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred))


## 7. Model Comparison and Visualization


In [ ]:
# Compare model performance
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results.keys()],
    'AUC-ROC': [results[m]['auc'] for m in results.keys()],
    'CV Accuracy': [results[m]['cv_mean'] for m in results.keys()],
    'CV Std': [results[m]['cv_std'] for m in results.keys()]
})

print("\nModel Comparison:")
print(comparison_df.to_string(index=False))

# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy comparison
sns.barplot(data=comparison_df, x='Model', y='Accuracy', ax=axes[0], palette='muted')
axes[0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Accuracy')
axes[0].tick_params(axis='x', rotation=45)

# AUC-ROC comparison
sns.barplot(data=comparison_df, x='Model', y='AUC-ROC', ax=axes[1], palette='muted')
axes[1].set_title('Model AUC-ROC Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('AUC-ROC')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../results/figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


## 8. ROC Curves


In [ ]:
# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['y_pred_proba'])
    plt.plot(fpr, tpr, label=f"{name} (AUC = {result['auc']:.3f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves for Different Models', fontsize=16, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()


## 9. Biomarker Identification


In [ ]:
# Identify top biomarkers based on feature importance
top_biomarkers = feature_importance.head(15)

print("\nTop 15 DNA Sequence Biomarkers:")
print("=" * 60)
for idx, row in top_biomarkers.iterrows():
    print(f"{row['feature']:30s}: Importance = {row['importance']:.4f}")

# Save biomarker list
import os
os.makedirs('../results', exist_ok=True)
top_biomarkers.to_csv('../results/top_biomarkers.csv', index=False)
print("\nBiomarker list saved to '../results/top_biomarkers.csv'")


## 10. Summary and Conclusions


In [ ]:
# Find best model
best_model_name = max(results.keys(), key=lambda x: results[x]['auc'])

print("\n" + "="*60)
print("ANALYSIS SUMMARY")
print("="*60)
print(f"\nTotal sequences analyzed: {len(df)}")
print(f"Total features extracted: {X.shape[1]}")
print(f"\nBest performing model: {best_model_name}")
print(f"Best AUC-ROC score: {results[best_model_name]['auc']:.4f}")
print(f"Best accuracy: {results[best_model_name]['accuracy']:.4f}")
print(f"\nTop 5 Biomarkers:")
for idx, row in top_biomarkers.head(5).iterrows():
    print(f"  - {row['feature']}: {row['importance']:.4f}")
print("\n" + "="*60)
print("\nKey Findings:")
print("1. DNA sequence features can effectively distinguish between")
print("   potential drug targets/biomarkers (label=1) and non-targets (label=0)")
print("2. Features such as GC content, nucleotide frequencies, and")
print("   k-mer patterns are highly predictive")
print("3. Machine learning models can identify important sequence")
print("   characteristics that may serve as biomarkers for drug targeting")
print("\n" + "="*60)
